In [25]:
# import classes from folder app/classes.py
from app.libs.classes import Shift, Nurse
import json


In [26]:
with open('data/shifts.json', 'r') as f:
    shifts = json.load(f)

shifts = {shift['shift_id']: Shift(**shift) for shift in shifts}

with open('data/nurses.json', 'r') as f:
    nurses = json.load(f)

nurses = {nurse['nurse_id']: Nurse(**nurse) for nurse in nurses}
nurses
# Additional parameters 

In [28]:
DM = 8
HMM = 216
# Modelo

In [29]:
from datetime import datetime, timedelta
from math import floor
from typing import Dict

from app.libs.classes import Shift, Nurse
from app.libs.constants import HMM, DM
import pulp as plp
from itertools import product

In [30]:
import pulp as plp

model = plp.LpProblem("Nurse Scheduling", plp.LpMinimize)
I = set(nurses.keys())
J = set(shifts.keys())
K = set([datetime.strptime(shifts[j].shift_date, '%Y-%m-%d').day for j in J])
week=set([datetime.strptime(shifts[j].shift_date, '%Y-%m-%d').isocalendar()[1] for j in J])

valid_keys = [(i, j) for (i,j) in product(I, J) if datetime.strptime(shifts[j].shift_date, '%Y-%m-%d').day not in nurses[i].vacations]


/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/pulp/pulp.py:1352: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")


In [31]:
X = plp.LpVariable.dicts("X", valid_keys, cat=plp.LpBinary)
W = plp.LpVariable.dicts("W", [(i, k) for (i,k) in product(I, K)], cat=plp.LpBinary)
Zmorning=plp.LpVariable.dicts("Zmorning", [(i, w) for (i,w) in product(I, week)], cat=plp.LpBinary)
Zafternoon=plp.LpVariable.dicts("Zafternoon", [(i, w) for (i,w) in product(I, week)], cat=plp.LpBinary)

V = plp.LpVariable.dicts("V", I, cat=plp.LpContinuous,lowBound=0)  # overtime
Y = plp.LpVariable.dicts("Y", I, cat=plp.LpContinuous,lowBound=0) # TOTAL NUMBER OF SHIFTS ASSIGNED TO NURSE i
MDH = plp.LpVariable("MDH", cat=plp.LpContinuous,lowBound=0) # MAXIMUM DIFFERENCE BETWEEN MAXIMUM SHIFTS AND REAL NUMBER OF SHIFTS ASSIGNED
DOff = plp.LpVariable.dicts("DOff", I, cat=plp.LpContinuous,lowBound=0) 


# Objective function

In [32]:
expr_PDL = plp.lpSum([(X[i, j] for (i,j) in valid_keys if datetime.strptime(shifts[j].shift_date, '%Y-%m-%d').day in nurses[i].dates_off)])
# accounts for the number of weekends a person is assigned to a shif not in his/her preference
expr_PWE = plp.lpSum([(X[i, j]) for (i,j) in valid_keys if
                        datetime.strptime(shifts[j].shift_date, '%Y-%m-%d').weekday() in [5, 6] and
                        (nurses[i].shift_preference != shifts[j].shift_type)])

Obj_2 = MDH + plp.lpSum([V[i] for i in I]) * (1 / DM)

In [33]:
# accounts the number of shifts assigned to a nurse
for nurse in I:
    model+=Y[nurse] == plp.lpSum([X[nurse, shift] for shift in J if (nurse, shift) in valid_keys]), f"assigned_shifts_{nurse}"

In [34]:
 # accounts for the difference between shifts that should be assigned to a caregiver 
            # (considering overtime hours to be balanced) and real number of shifts assigned

for nurse in I:
    expr = Y[nurse] - (HMM - nurses[nurse].accumulated_hours) * (1 / DM) <= MDH
    model += expr, f"balance_hours_1_{nurses[nurse].nurse_id}"

    expr = (HMM - nurses[nurse].accumulated_hours) * (1 / DM) - Y[nurse]<= MDH
    model += expr,  f"balance_hours_2_{nurses[nurse].nurse_id}"
    

In [35]:
# dictionary enumerating the shifts happening in each day
shifts_per_day = {}
for shift in J:
    if shifts[shift].shift_date not in shifts_per_day:
        shifts_per_day[shifts[shift].shift_date] = []
    shifts_per_day[shifts[shift].shift_date].append(shift)


In [36]:
for i in I:
    for spd in shifts_per_day.keys():
        model += plp.lpSum([X[i, j] for j in shifts_per_day[spd] if (i, j) in valid_keys]) <= 1, f"one_shift_per_day_{nurses[i].nurse_id}_{spd}"


In [37]:
# accounts for overtime V hours required
for i in I:
    model += Y[i] * DM <= (HMM - nurses[i].accumulated_hours) + V[i], f"overtime_{nurses[i].nurse_id}"

In [38]:
# demand is satisfied
for j in J:
    model += plp.lpSum([X[(i, j)] for i in I if (i, j) in valid_keys]) >= shifts[j].demand, f"demand_{j}"
        

In [39]:
# for each weekend, a nurse works at most one day
for i in I:
    for j1 in J:
        if (i, j1) in valid_keys and datetime.strptime(shifts[j1].shift_date,'%Y-%m-%d').weekday() == 5:
            model += X[i, j1] + plp.lpSum([X[i, j2] for j2 in J if
                                                        (i, j2) in valid_keys and datetime.strptime(shifts[j2].shift_date, '%Y-%m-%d').weekday() == 6
                                                        and datetime.strptime(shifts[j2].shift_date,'%Y-%m-%d') == datetime.strptime(shifts[j1].shift_date, '%Y-%m-%d') + timedelta(days=1)]) <= 1, f"weekend_{nurses[i].nurse_id}_{j1}"


In [40]:
for i,k in product(I, K):
    model += (1 - plp.lpSum([X[i, j] for j in J if (i, j) in valid_keys and datetime.strptime(shifts[j].shift_date, '%Y-%m-%d').day == k])) ==W[i, k], f"day_is_off_{nurses[i].nurse_id}_{k}"

In [41]:
days = list([datetime.strptime(shifts[j].shift_date, '%Y-%m-%d').day for j in J if datetime.strptime(shifts[j].shift_date, '%Y-%m-%d').weekday() in [5, 6]])

total_weekend_days = len(days)

for i in I:
    model += DOff[i] == plp.lpSum([W[i, k] for k in days]), f"weekend_off_1_{nurses[i].nurse_id}"
    model += DOff[i] >= floor(total_weekend_days / 2) + 1, f"weekend_off_2_{nurses[i].nurse_id}"

In [42]:
for i,w in product(I, week):
    model += Zmorning[i, w] + Zafternoon[i, w] <= 1, f"shifts_per_week_{nurses[i].nurse_id}_{w}"

In [43]:
# if a nurse works in a morning shift the varriable Z_morning is equal to 1
for i in I:
    for j in J:
        if (i, j) in valid_keys and shifts[j].shift_type == 'morning':
            model +=  X[i, j]<=Zmorning[i, datetime.strptime(shifts[j].shift_date, '%Y-%m-%d').isocalendar()[1]], f"morning_shift_{nurses[i].nurse_id}_{j}"
        elif (i, j) in valid_keys and shifts[j].shift_type == 'afternoon':
            model +=  X[i, j]<=Zafternoon[i, datetime.strptime(shifts[j].shift_date, '%Y-%m-%d').isocalendar()[1]], f"afternoon_shift_{nurses[i].nurse_id}_{j}"

In [44]:
#an afternoon shift cannot be followed by a morning shift
for i in I:
    for j in J:
        if (i, j) in valid_keys and shifts[j].shift_type == 'morning':
            model +=  X[i, j] + plp.lpSum([X[i, j2] for j2 in J if
                                                        (i, j2) in valid_keys and datetime.strptime(shifts[j2].shift_date, '%Y-%m-%d') == datetime.strptime(shifts[j].shift_date, '%Y-%m-%d') - timedelta(days=1)
                                                        and shifts[j2].shift_type == 'afternoon']) <= 1, f"morning_afternoon_{nurses[i].nurse_id}_{j}"

In [45]:
#
model.setObjective(expr_PDL+expr_PWE)


In [46]:
# solve the problem using HIGHS
solver = plp.GUROBI_CMD(msg=0)
model.solve(solver)
#Obj_2 = MDH + plp.lpSum([V[i] for i in I]) * (1 / DM)


1

In [47]:
print(model.objective.value())

1.0


In [48]:

objective = model.objective.value()
epsilon = 1
model+= expr_PDL+expr_PWE<=objective + epsilon , "value_obj_1"
model.setObjective(Obj_2)
model.solve()
print(model.objective.value(),objective)
while model.status == 1:
    model.solve()
    print(expr_PDL.value()+expr_PWE.value(),Obj_2.value())
    epsilon += 1
    ctr = model.constraints["value_obj_1"]
    ctr.changeRHS(objective + epsilon)


    if objective + epsilon ==7:
        break
    
model.setObjective(Obj_2)

4.25 1.0
2.0 4.25
3.0 4.0
4.0 3.25
5.0 3.25
6.0 3.25


In [49]:
#Get gurobi model status code
#set gurobi verbose to 1
solver = plp.GUROBI_CMD(msg=1)
model.solve(solver)

Set parameter Username
Set parameter LogFile to value "gurobi.log"
Academic license - for non-commercial use only - expires 2024-04-04
Using license file /Users/user/gurobi.lic

Gurobi Optimizer version 10.0.1 build v10.0.1rc0 (mac64[x86])
Copyright (c) 2023, Gurobi Optimization, LLC

Read LP format model from file /var/folders/xc/t5gbnt3n5sq5bkg65xw9f1g00000gn/T/0702a4e54de347d49fcdb0c939442107-pulp.lp
Reading time = 0.01 seconds
OBJ: 3705 rows, 2166 columns, 16056 nonzeros

CPU model: Intel(R) Core(TM) i7-6820HQ CPU @ 2.70GHz
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 3705 rows, 2166 columns and 16056 nonzeros
Model fingerprint: 0x9d64a85e
Variable types: 32 continuous, 2134 integer (2134 binary)
Coefficient statistics:
  Matrix range     [1e+00, 8e+00]
  Objective range  [1e-01, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+02]
Presolve removed 1976 rows and 697 columns
Presolve time: 0.03s
Presolved: 1

1

In [50]:
import pandas as pd
import numpy as np
for i in I:
    for j in J:
        if (i, j) in valid_keys and X[i, j].varValue > 0.5:
            if datetime.strptime(shifts[j].shift_date, '%Y-%m-%d').day in [1,7,8,14,15,21,22,28,29] and shifts[j].shift_type!=nurses[i].shift_preference:
                print(f"Nurse {nurses[i].nurse_name} works in shift {shifts[j].shift_id} on {shifts[j].shift_date} ({shifts[j].shift_type}--{shifts[j].shift}--{nurses[i].shift_preference}) with vacation {nurses[i].vacations} and days off {nurses[i].dates_off}.")
            if datetime.strptime(shifts[j].shift_date, '%Y-%m-%d').day in nurses[i].dates_off:
                print(f"Nurse {nurses[i].nurse_name} works in shift {shifts[j].shift_id} on {shifts[j].shift_date} ({shifts[j].shift_type}--{shifts[j].shift}--{nurses[i].shift_preference}) with vacation {nurses[i].vacations} and days off {nurses[i].dates_off}.")

date_range=np.sort(list(set([shifts[j].shift_date for j in range(len(shifts))])))
nurses_shifts = {nurses[i].nurse_name: {shifts[j].shift_date:shifts[j].shift+" - "+shifts[j].shift_type for j in J if (i,j) in valid_keys and X[i, j].varValue > 0.5} for i in I}          

tabulated_shifts=[]
for nurse in nurses_shifts:
    line=[nurse]
    for date in date_range:
        if date not in nurses_shifts[nurse]:
            line.append("-")
        else:
           line.append(nurses_shifts[nurse][date])
    tabulated_shifts.append(line)
tabulated_shifts=pd.DataFrame(tabulated_shifts, columns=["Nurse"]+[date for date in date_range])

Nurse Nurse 2 works in shift 42 on 2022-05-08 (morning--M--afternoon) with vacation [] and days off [21, 29].
Nurse Nurse 3 works in shift 87 on 2022-05-15 (afternoon--T1--afternoon) with vacation [] and days off [14, 8, 29, 1, 28, 15, 7].
Nurse Nurse 4 works in shift 123 on 2022-05-21 (afternoon--T1--afternoon) with vacation [7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20] and days off [28, 8, 21, 14, 29, 7, 22, 1].
Nurse Nurse 4 works in shift 165 on 2022-05-28 (afternoon--T1--afternoon) with vacation [7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20] and days off [28, 8, 21, 14, 29, 7, 22, 1].
Nurse Nurse 5 works in shift 80 on 2022-05-14 (morning--DISP--afternoon) with vacation [30, 31] and days off [21, 15].
Nurse Nurse 7 works in shift 0 on 2022-05-01 (morning--M--morning) with vacation [17, 18, 19, 20, 21, 22, 23, 24, 25, 26] and days off [8, 1, 14, 22, 28].
Nurse Nurse 9 works in shift 126 on 2022-05-22 (morning--M--afternoon) with vacation [] and days off [8].


In [52]:

tabulated_shifts

,Nurse,2022-05-01,2022-05-02,2022-05-03,2022-05-04,2022-05-05,2022-05-06,2022-05-07,2022-05-08,2022-05-09,...,2022-05-22,2022-05-23,2022-05-24,2022-05-25,2022-05-26,2022-05-27,2022-05-28,2022-05-29,2022-05-30,2022-05-31
0,Nurse 0,-,M - morning,-,M - morning,M - morning,M - morning,DISP - morning,-,-,...,-,M - morning,-,M - morning,M - morning,COM - morning,-,M - morning,-,-
1,Nurse 1,DISP - morning,M - morning,-,COM - morning,-,-,-,DISP - morning,T1 - afternoon,...,DISP - morning,M - morning,M - morning,M - morning,M - morning,COM - morning,M - morning,-,T1 - afternoon,T1 - afternoon
2,Nurse 2,T1 - afternoon,-,M - morning,-,M - morning,-,-,M - morning,CHX - afternoon,...,T1 - afternoon,T1 - afternoon,-,-,-,T1 - afternoon,-,-,-,T2 - afternoon
3,Nurse 3,-,T2 - afternoon,-,T2 - afternoon,-,CHX - afternoon,-,-,T1 - afternoon,...,T1 - afternoon,-,COM - morning,COM - morning,COM - morning,M - morning,-,-,-,CHX - afternoon
4,Nurse 4,-,T1 - afternoon,T1 - afternoon,CHX - afternoon,T2 - afternoon,T1 - afternoon,-,-,-,...,-,T2 - afternoon,T2 - afternoon,T2 - afternoon,T1 - afternoon,CHX - afternoon,T1 - afternoon,-,T1 - afternoon,T2 - afternoon
5,Nurse 5,T1 - afternoon,CHX - afternoon,-,-,T2 - afternoon,T2 - afternoon,-,T1 - afternoon,-,...,-,CHX - afternoon,T1 - afternoon,T2 - afternoon,T2 - afternoon,T2 - afternoon,-,T1 - afternoon,-,-
6,Nurse 6,DISP - morning,T1 - afternoon,T2 - afternoon,T1 - afternoon,CHX - afternoon,-,-,-,T2 - afternoon,...,-,-,CHX - afternoon,CHX - afternoon,CHX - afternoon,T2 - afternoon,-,-,-,-
7,Nurse 7,M - morning,COM - morning,COM - morning,M - morning,COM - morning,M - morning,M - morning,-,COM - morning,...,-,-,-,-,-,M - morning,-,DISP - morning,DISP - morning,M - morning
8,Nurse 8,-,M - morning,M - morning,COM - morning,-,COM - morning,-,-,M - morning,...,-,COM - morning,M - morning,M - morning,M - morning,-,DISP - morning,-,COM - morning,COM - morning
9,Nurse 9,T1 - afternoon,CHX - afternoon,CHX - afternoon,-,T1 - afternoon,-,T1 - afternoon,-,M - morning,...,M - morning,-,CHX - afternoon,T1 - afternoon,-,-,T1 - afternoon,-,M - morning,M - morning
